In [ ]:
!pip install --upgrade --no-cache-dir gdown

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14774 sha256=fcac43789d8f36fa9fe443e25e4b036f4bd77a3b74e9d89cd94176c285dbdc91
  Stored in directory: /tmp/pip-ephem-wheel-cache-fd281xc3/wheels/fb/c3/0e/c4d8ff8bfcb0461afff199471449f642179b74968c15b7a69c
Successfully built gdown
  Attempting uninstall: gdown
    Found existing installation: gdown 4.2.2
    Uninstalling gdown-4.2.2:
      Successfully uninstalled gdown-4.2.2


In [ ]:
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [ ]:
import gdown
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt 
%matplotlib inline 
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit

#NLTK
import nltk
from nltk.corpus import stopwords
#nltk.download("english")
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
pd.set_option('display.max_columns', None)

**Задаа 0: Загрузим и предобработаем данные**

In [ ]:
!gdown 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ

Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 59.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:05<00:00, 122MB/s]


**movies_metadata.csv:** The main Movies Metadata file. Contains information on 45,000 movies featured in the Full MovieLens dataset. Features include posters, backdrops, budget, revenue, release dates, languages, production countries and companies.

**ratings.csv:** Ratings of the Full Dataset.

In [ ]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [ ]:
# удалим ошибочные значения

movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [ ]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [ ]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

In [ ]:
# оставим только колонки, которые мы собираемся использовать

movies_metadata = movies_metadata[['id', 'original_title', 'overview']]

In [ ]:
# заполняем пустые значения

movies_metadata = movies_metadata.fillna('')

In [ ]:
# удалим дубли, потому что они есть и очень мешаются

movies_metadata = movies_metadata.drop_duplicates().reset_index(drop=True)

In [ ]:
movies_metadata = movies_metadata.rename(columns={'id':'movieId'})

**1 Получим случайную матрицу эмбедингов для каждого фильма**

In [ ]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    '''
       на вход: 
       - data - датафрейм с описанием и названием фильмов
       - lemmatizer - класс для лемматизации текстов
       - stop_words - стоп-слова
      
       на выходе: 
       - список с предобработанными текстовыми данными
    '''
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]

        # приводим к нижнему регистру и дропаем стоп-слова
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [ ]:
nltk.download("stopwords")

text_overview = preprocessing(movies_metadata["overview"])
text_title = preprocessing(movies_metadata["original_title"])

movies_metadata["overview"] = text_overview
movies_metadata["original_title"] = text_title 

# закидываем предобработанные описания и названия фильмов в общий список

text_prepr = text_title + text_overview

text_prepr = [i.split() for i in text_prepr]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/45433 [00:00<?, ?it/s]

  0%|          | 0/45433 [00:00<?, ?it/s]

In [ ]:
# word2vec

model = Word2Vec(text_prepr, 
                 min_count=2)

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 42212


In [ ]:
# токенизация и эмбединг

tokenizer = Tokenizer()

# обновляем внутренний словарь на основе списка текстов

tokenizer.fit_on_texts(text_prepr)
vocab_size = len(tokenizer.word_index) + 1

print("Total words:", vocab_size)

Total words: 77765


In [ ]:
def embedding_matrix(tock, model):
    
    """
       На вход: словарь tokenizer.word_index 
       модель word2vec
        
       На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 300))

    for word, i in tock.items():
        if word in model:
            embedding_matrix[i] = model[word].mean()

    return embedding_matrix

In [ ]:
id_film = movies_metadata['movieId'].nunique()

matrix_emb = embedding_matrix(tokenizer.word_index, model.wv)

matrix_emb = pd.concat([movies_metadata[['movieId']],
                    pd.DataFrame(matrix_emb)], axis=1)

In [ ]:
matrix_emb.shape

(45433, 301)

In [ ]:
matrix_emb.head(2)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,862,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,8844,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [ ]:
# из исходных данных берем айдишник и добавляю 300 случайных колонок

#matrix_emb = pd.concat([movies_metadata[['id']], 
#                        pd.DataFrame(np.random.random((movies_metadata.shape[0], 300)))], 
#                       axis=1)

In [ ]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей

matrix_emb['movieId'] = matrix_emb['movieId'].astype(int)

**2 Теперь напишем функцию train_test_split по юзеру по матрице рейтингов (юзер х фильмы = рейтинг)**

**Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях**


In [ ]:
ratings = pd.read_csv('ratings.csv')

In [ ]:
ratings.sample(5)

,userId,movieId,rating,timestamp
7640207,78887,4343,4.0,1043259079
2846765,29675,2353,4.5,1463369948
21793344,226332,40583,4.0,1240194978
2969788,30954,2002,4.0,1099012566
25452621,264990,762,3.0,1152408517


In [ ]:
# оказвается, только к 7565 фильмам из movies_metadata есть оценки в ratings

common_ids = set(matrix_emb.movieId).intersection(set(ratings.movieId))

len(common_ids)

7565

In [ ]:
# выберем только пересекающиеся фильмы

#matrix_emb = matrix_emb[matrix_emb.id.isin(common_ids)]

In [ ]:
#matrix_emb.shape

In [ ]:
ratings = ratings[ratings.movieId.isin(common_ids)]

In [ ]:
len(ratings.movieId.unique())

7565

In [ ]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # отфильтровываем пользователей с количеством просмотров < 10
    if temp_df.shape[0] < 10:
      continue

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [ ]:
# вторым параметром на вход можно подать любой список пользователей

splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:10], 0.7)

**3 Для каждого юзера по train-фильмам находим средний вектор эмбединг через mean**

**Надо дополнительно домножить эмбединг-вектор на оценку фильма**

In [ ]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['movieId'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['movieId'])
                                  .reset_index()
                                  .rename(columns={'index':'rating'}), on='movieId')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['movieId'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .sum()
                              .mul(1 / temp_df['rating'].sum(), axis=0))
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [ ]:
# получим матрицу пользователей

res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [ ]:
# преобразуем к табличному виду

res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'movieId'})

In [ ]:
res_vec_df.head(2)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,1,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232,0.009232
1,2,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.012409,0.01

**Задача 4: Найдем топ близких фильмов, которые пользователь еще не смотрел (косинусное расстояние)**

In [ ]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, user_data_dict, top_n=10):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['movieId'] == user_id] \
                                .drop('movieId', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['movieId']:
    
    film_vec = films_emb_mtrx[films_emb_mtrx['movieId'] == film] \
                              .drop('movieId', axis=1) \
                              .values \
                              .flatten() \
                              .tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in user_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  # записываем их в пятый список под айди пользователя в словаре с train-test-data
  user_data_dict[user_id].append(heapq.nlargest(top_n, scores, key=scores.get))

In [ ]:
# получаем предсказания для пользователей

for i, user in enumerate(splitted_data.keys()):
  print('Running:', i + 1, '/', len(splitted_data))
  get_closest_films(res_vec_df, user, matrix_emb, splitted_data)

Running: 1 / 8


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Running: 2 / 8
Running: 3 / 8
Running: 4 / 8
Running: 5 / 8
Running: 6 / 8
Running: 7 / 8
Running: 8 / 8


**Задача 5: Проверить результаты по метрикам**

In [ ]:
def mean_precision(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика precision@k по всем пользователям
  '''

  sum_precision = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    k = len(user_data_dict[user][4])
    
    sum_precision += TP / k

  # находим среднее по пользователям
  return sum_precision / len(user_data_dict)

In [ ]:
def mean_recall(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика recall@k по всем пользователям
  '''

  sum_recall = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    TP_FN = len(user_data_dict[user][2])
    
    sum_recall += TP / TP_FN

  # находим среднее по пользователям
  return sum_recall / len(user_data_dict)

In [ ]:
def MRR(user_data_dict):

  '''
     RR - обратный ранг, 1 / индеск первого вхождения рекомендации в тест

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MRR@k по всем пользователям
  '''

  sum_rr = 0

  # находим порядковый номер первого вхождения правильной рекомендации в тест
  for user in user_data_dict.keys():
    rank = [1 + splitted_data[1][4].index(x) for x in \
            set(splitted_data[1][4]).intersection(set(splitted_data[1][2]))]
    
    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
      rr = 0
    else:
      rr = 1 / min(rank)

    sum_rr += rr

  # находим среднее по пользователям
  return sum_rr / len(user_data_dict)

In [ ]:
def mean_average_precision_at_k(user_data_dict):

  '''
     Mean Average Precision - средняя точность по пользователям

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MAP@k по всем пользователям
  '''

  sum_ap = 0

  # итерируемся по пользователям
  for user in user_data_dict.keys():

    num_hits = 0
    score = 0

    for i, p in enumerate(user_data_dict[user][4]):

      if p in user_data_dict[user][2] and p not in user_data_dict[user][4][:i]:
        num_hits += 1
        score += num_hits / (i + 1)

    sum_ap += score / min(len(user_data_dict[user][2]), len(user_data_dict[user][4]))

  return sum_ap / len(user_data_dict)

In [ ]:
print('Mean Precision: ', mean_precision(splitted_data))
print('Mean Recall: ', mean_recall(splitted_data))
print('Mean Reciprocal Rank:', MRR(splitted_data))
print('Mean Average Precision at K:', mean_average_precision_at_k(splitted_data))

Mean Precision:  0.0
Mean Recall:  0.0
Mean Reciprocal Rank: 0.0
Mean Average Precision at K: 0.0
